In [ ]:
%matplotlib inline

In [ ]:
true_path = "/Users/noah/workspace/research/uw-machine-learning/data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/"
    

In [ ]:
import re

def open_caseid(caseid):
    out_3d_path = "/Users/noah/workspace/models/SAMUWgh/OUT_3D"
    out_2d_path = "/Users/noah/workspace/models/SAMUWgh/OUT_2D"
   
    d3 = xr.open_mfdataset(f"{out_3d_path}/{caseid}_*.nc", autoclose=True)
    d2 = xr.open_mfdataset(f"{out_2d_path}/{caseid}_*.nc")
    
    return d2, d3


def open_caseid_3d(caseid):
    out_3d_path = "/Users/noah/workspace/models/SAMUWgh/OUT_3D"   
    ds = xr.open_mfdataset(f"{out_3d_path}/{caseid}_*.nc", autoclose=True)
    return ds


def swap_z_for_pres(ds):
    ds['p'] = ds.p[0].drop('time')
    return ds.swap_dims({'z': 'p'})


def list_caseids():
    """List the available caseids"""
    ncs = !ls ../OUT_3D/*.nc
    pat = re.compile("^../OUT_3D/(.*?)_\d_\d+.nc$")
    return set(pat.search(nc).group(1) for nc in ncs)


def plot_zonal_velocity(mean):
    im = plt.contourf(mean.y, mean.p, mean,
                      levels=np.r_[-40:45:5], cmap='RdBu_r',
                      extend='both')


    plt.contour(mean.y, mean.p, mean, levels=[-10, -8,-6,-4,-2,0], colors='k')
    plt.ylim([1000,10])
    plt.colorbar(im)
    plt.title("Zonal Velocity")
    plt.xlabel("y")
    plt.ylabel("p");


In [ ]:
list_caseids()

In [ ]:
d2, d3 = open_caseid("NGAqua_ngaqua.dt30")

# Climate

In [ ]:
mean = swap_z_for_pres(d3).mean(['time', 'x'])

In [ ]:
p = mean.p
tm = mean.TABS

tht  = tm * (1000/p)**(2/7)

levels = np.r_[270:350:5]
im = tht.plot.contourf(x='y', y='p', levels=levels, extend='both')
plt.clabel(im, colors='black', inline=False, fmt="%.0f")
plt.ylim([1000, 10])
plt.title("Potential Temperature")
plt.xlabel("y")
plt.ylabel("p")
# tht.plot.contourf(levels=levels)

In [ ]:
plt.contourf(tm.y, p, tm)
plt.ylim([1000,10])
plt.colorbar()
plt.title("Absolute Temperature")
plt.xlabel("y")
plt.ylabel("p");

In [ ]:
plot_zonal_velocity(mean.U)

In [ ]:
im = plt.contourf(mean.y, mean.p, mean.V, cmap='RdBu_r',
                  levels=np.r_[-30:35:5])

plt.ylim([1000,10])
plt.colorbar(im)
plt.title("Meridional Velocity")
plt.xlabel("y")
plt.ylabel("p");

In [ ]:
ic = xr.open_dataset("../NGAqua/ic.nc")
ic.V[:,:,15].plot()

# Spin-up

Now I plot the temporal evolution of some different fields at a lower atmosphere height as the simulation spins up

Here is the zonal velocity

In [ ]:
d3.U[::4, 5].plot(col='time', col_wrap=4)

and the meridional velocity

In [ ]:
d3.V[::4, 5].plot(col='time', col_wrap=4)

and the absolute temperature

In [ ]:
d3.TABS[::4, 10].plot(col='time', col_wrap=4)

Why does the model seem to setup a N-S temperature gradient? Why do easterly flows above the equator set in?

# What happens if I turn off all physics?

In [ ]:
d3 = open_caseid_3d("NGAqua_dt30.noPhys")

In [ ]:
d3.U[::4,5].plot(col='time', col_wrap=4)

In [ ]:
d3.TABS[::4,2].plot(col='time', col_wrap=4)

This issue with the boundaries is also present in the noPhys run. 


Chris thinks this issue might be due to the staggering of the meridional velocity field. This problem can be diagnosed by seeing how much the velocity field changes after a very small time step. 

In [ ]:
def test_dataset(caseid):
    paths = [f"../OUT_3D/{caseid}_1_0000000000.nc", 
             f"../OUT_3D/{caseid}_1_0000000001.nc"]
    return xr.open_mfdataset(paths, concat_dim='time').assign_coords(time=[0,1])

ds = test_dataset("NGAqua_test.ic")

In [ ]:

ds.U[0,5].plot()

In [ ]:
ds.U[1,5].plot()

The zonal velocity looks pretty similar after this number of steps, so let's now look at the meridional velocity equation.

In [ ]:
def plot_v_slice(ds):

    ds.V[0,5,:,0].plot(label='T0', marker='.')
    ds.V[1,5,:,0].plot(label='T1', marker='o', alpha=.5)

    plt.legend()

In [ ]:
plot_v_slice(ds)

As we can see there is a big discrepency in the northern part of the domain after a single 1e-8 second time step. This implies that we should check the velocity staggering. What is the velocity at the southern and northern boundaries?

In [ ]:
ds.V.isel(y=0).plot(col='time')

In [ ]:
ds.V.isel(y=-1).plot(col='time')

# Interface initial conditions

The meridional velocity at the northern boundary is much larger than at the southern boundary. I need to be careful about the staggering of these variables. Instead, let's initialize the model with the data averaged along the interfaces.

In [ ]:
ds_stag = test_dataset("NGAqua_test.ic.staggered")
plot_v_slice(ds_stag)

We can see that the V velocity is essentially unchanged by the pressure projection.

In [ ]:
np.abs(ds_stag.diff('time')).mean().load()

More quantitiavely, we can see that the MAD of the velocities from between the time steps is very small. Therefore, we should initialize the model with the data from the interfaces. Let's now try, running a dry dynamics simulation with this new initial condition for longer.

In [ ]:
ds = open_caseid_3d("NGAqua_icStag.dt30").pipe(swap_z_for_pres)

In [ ]:
ds.U[::8,5].plot(col='time', col_wrap=3)

In [ ]:
ds.V[::8,5].plot(col='time', col_wrap=3)

In [ ]:
v = 'TABS'

anom = ds[v] - ds[v][0].mean('x')
anom[::4, 5].plot(col='time', col_wrap=3)

The simulations are much improved near the boundaries, but the data develops a **grainy appearance** after a few days. I would expect it to get smoother as time progesses since this is a coarse resolution model. Is this caused by the SGS scheme or some other problem? Is the initial data too rough somehow? Is it related to the north/south boundary conditions?

## With SGS

In [ ]:
ds = open_caseid_3d('NGAqua_icStag.SGS.dt30')

In [ ]:
v = 'TABS'

anom = ds[v] - ds[v][0].mean('x')
anom[::4, 5].plot(col='time', col_wrap=3)

This looks exactly the same. maybe there is some mistake.

## With Microphysics, Radiation, etc

Now, lets turn on the rest of the physics packages in SAM, and see what the solutions looks like.

In [ ]:
ds = open_caseid_3d('NGAqua_icStag.SGS.MICRO.RAD.dt30')

In [ ]:
v = 'TABS'

anom = ds[v] - ds[v][0].mean('x')
anom[::4, 5].plot(col='time', col_wrap=3)

Turning on the various physics packages makes it work better. The graininess is from the grid-scale storms in the SAM model.